In [11]:
import qrcode
import os # Para construir rutas de archivo de forma segura

# --- Importaciones de ReportLab ---
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.lib.utils import ImageReader # Para manejar imágenes

# --- Lista de direcciones y sus enlaces Waze (igual que antes) ---
direcciones = [
    # CODELCO
    {"empresa": "CODELCO", "nombre_base_archivo": "qr_codelco_coya_h267", "direccion_original": "H-267 48, Coya, Machalí, O'Higgins", "enlace_waze": "https://waze.com/ul?ll=-34.0875,-70.5062&navigate=yes", "coordenadas": "-34.0875, -70.5062 (aprox. Tranques Barahona)"},
    {"empresa": "CODELCO", "nombre_base_archivo": "qr_codelco_saladillo_los_andes", "direccion_original": "Saladillo, Los Andes, Valparaíso", "enlace_waze": "https://waze.com/ul?ll=-32.9264,-70.2600&navigate=yes", "coordenadas": "-32.9264, -70.2600 (aprox.)"},
    {"empresa": "CODELCO", "nombre_base_archivo": "qr_codelco_garita_tove_tiltil", "direccion_original": "Garita Acceso TOVE, Tiltil, Región Metropolitana (Depósito Relaves Ovejería)", "enlace_waze": "https://waze.com/ul?ll=-33.041578,-70.798303&navigate=yes", "coordenadas": "-33.041578, -70.798303"},
    {"empresa": "CODELCO", "nombre_base_archivo": "qr_codelco_embalse_caren_alhue", "direccion_original": "Embalse Carén, Alhué, Región Metropolitana", "enlace_waze": "https://waze.com/ul?ll=-34.107167,-71.139306&navigate=yes", "coordenadas": "-34.107167, -71.139306"},
    {"empresa": "CODELCO", "nombre_base_archivo": "qr_codelco_control_maitenes_machali", "direccion_original": "Control 2 Maitenes, Machalí, O'Higgins", "enlace_waze": "https://waze.com/ul?ll=-34.193850,-70.563186&navigate=yes", "coordenadas": "-34.193850, -70.563186 (Acceso Principal)"},
    {"empresa": "CODELCO", "nombre_base_archivo": "qr_codelco_quintero_puchuncavi", "direccion_original": "Quintero, Puchuncaví, Valparaíso (Codelco Ventanas)", "enlace_waze": "https://waze.com/ul?ll=-32.7470,-71.4842&navigate=yes", "coordenadas": "-32.7470, -71.4842 (aprox.)"},
    # PELAMBRES
    {"empresa": "PELAMBRES", "nombre_base_archivo": "qr_pelambres_shell_las_esteras_quilicura", "direccion_original": "Shell Las Esteras, Panamericana Norte 372, Quilicura, Región Metropolitana", "enlace_waze": "https://waze.com/ul?ll=-33.3645,-70.7075&navigate=yes", "coordenadas": "-33.3645, -70.7075 (aprox. dirección, verificar Shell)"},
    {"empresa": "PELAMBRES", "nombre_base_archivo": "qr_pelambres_shell_hijuelas_km108", "direccion_original": "Shell Hijuelas - Panamericana Norte Km. 108, Quillota Hijuelas, Valparaíso, Chile", "enlace_waze": "https://waze.com/ul?ll=-32.7982,-71.1591&navigate=yes", "coordenadas": "-32.7982, -71.1591"},
    {"empresa": "PELAMBRES", "nombre_base_archivo": "qr_pelambres_copec_la_calera", "direccion_original": "Copec, La Calera (Copec Carrera)", "enlace_waze": "https://waze.com/ul?ll=-32.791633,-71.202731&navigate=yes", "coordenadas": "-32.791633, -71.202731"},
    {"empresa": "PELAMBRES", "nombre_base_archivo": "qr_pelambres_copec_hijuelas_km10", "direccion_original": "COPEC HIJUELAS Carretera Panamericana Norte Km. 10 Ruta 5 Oriente, Hijuelas, Valparaíso", "enlace_waze": "https://waze.com/ul?ll=-32.8713,-71.0855&navigate=yes", "coordenadas": "-32.8713, -71.0855 (aprox.)"}
]

# --- Directorio para guardar los QRs (opcional, pero buena práctica) ---
output_dir = "qrcodes_waze"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- Generar y guardar cada código QR ---
print("Generando códigos QR...")
for item in direcciones:
    img = qrcode.make(item["enlace_waze"])
    # Guardar en el subdirectorio
    item["ruta_completa_qr"] = os.path.join(output_dir, item["nombre_base_archivo"] + ".png")
    img.save(item["ruta_completa_qr"])
    print(f"Código QR guardado como: {item['ruta_completa_qr']} (para: {item['direccion_original']})")
print("Proceso de generación de códigos QR completado.\n")


# --- Función para crear el PDF con los códigos QR ---
def create_pdf_with_qrcodes(pdf_filename="direcciones_waze_qr.pdf", data_list=None):
    if data_list is None:
        data_list = []

    print(f"Creando PDF: {pdf_filename}...")
    c = canvas.Canvas(pdf_filename, pagesize=letter)
    width, height = letter # Ancho y alto de la página

    # Márgenes
    margin_left = 0.75 * inch
    margin_top = 0.75 * inch
    margin_bottom = 0.75 * inch

    # Posición inicial (desde la parte superior, considerando el margen)
    y_position = height - margin_top

    # Tamaño del QR en el PDF
    qr_size = 1.5 * inch
    # Espacio entre elementos
    line_height_small = 0.2 * inch
    line_height_medium = 0.3 * inch
    space_after_qr = 0.5 * inch

    for i, item in enumerate(data_list):
        # Altura estimada para este bloque de item
        block_height = (line_height_medium * 2) + (line_height_small * 2) + qr_size + space_after_qr

        # Verificar si el bloque cabe en la página actual
        if y_position - block_height < margin_bottom:
            c.showPage() # Nueva página
            y_position = height - margin_top # Reiniciar Y a la parte superior

        # 1. Nombre de la empresa
        c.setFont("Helvetica-Bold", 12)
        y_position -= line_height_medium
        c.drawString(margin_left, y_position, f"Empresa: {item['empresa']}")

        # 2. Dirección Original
        c.setFont("Helvetica", 10)
        y_position -= line_height_medium
        # Manejo de texto largo para dirección (simple, puede mejorarse con Paragraphs)
        if len(item['direccion_original']) > 80: # Límite arbitrario
            c.drawString(margin_left, y_position, item['direccion_original'][:80] + "...")
        else:
            c.drawString(margin_left, y_position, item['direccion_original'])

        # 3. Coordenadas
        y_position -= line_height_small
        c.drawString(margin_left, y_position, f"Coords: {item['coordenadas']}")

        # 4. Enlace Waze (opcional, puede ocupar mucho espacio)
        # y_position -= line_height_small
        # c.setFont("Helvetica-Oblique", 8)
        # c.drawString(margin_left, y_position, f"Link: {item['enlace_waze']}")
        # c.setFont("Helvetica", 10) # Volver a fuente normal

        # 5. Código QR
        y_position -= (qr_size + line_height_small) # Espacio antes del QR y altura del QR
        qr_image_path = item["ruta_completa_qr"]

        try:
            # Usar ImageReader para mejor compatibilidad con reportlab
            qr_img_reader = ImageReader(qr_image_path)
            c.drawImage(qr_img_reader, margin_left, y_position, width=qr_size, height=qr_size, preserveAspectRatio=True, anchor='nw')
        except Exception as e:
            print(f"Error al dibujar imagen {qr_image_path} en el PDF: {e}")
            error_y_pos = y_position + (qr_size / 2) # Centrar el mensaje de error donde estaría el QR
            c.drawString(margin_left, error_y_pos, f"Error al cargar QR: {os.path.basename(qr_image_path)}")

        # Espacio después del bloque del QR
        y_position -= space_after_qr

        # Línea divisoria (opcional)
        if i < len(data_list) - 1 : # No después del último item
             if y_position - (0.1 * inch) > margin_bottom: # Solo si hay espacio
                c.line(margin_left, y_position, width - margin_left - (0.5*inch) , y_position) # Dibujar linea
                y_position -= (0.2 * inch) # Espacio después de la linea

    c.save()
    print(f"PDF generado como: {pdf_filename}")

# --- Llamar a la función para crear el PDF ---
create_pdf_with_qrcodes(pdf_filename="CODELCO_PELAMBRES_QR_WAZE.pdf", data_list=direcciones)

print("\nScript finalizado.")

Generando códigos QR...
Código QR guardado como: qrcodes_waze/qr_codelco_coya_h267.png (para: H-267 48, Coya, Machalí, O'Higgins)
Código QR guardado como: qrcodes_waze/qr_codelco_saladillo_los_andes.png (para: Saladillo, Los Andes, Valparaíso)
Código QR guardado como: qrcodes_waze/qr_codelco_garita_tove_tiltil.png (para: Garita Acceso TOVE, Tiltil, Región Metropolitana (Depósito Relaves Ovejería))
Código QR guardado como: qrcodes_waze/qr_codelco_embalse_caren_alhue.png (para: Embalse Carén, Alhué, Región Metropolitana)
Código QR guardado como: qrcodes_waze/qr_codelco_control_maitenes_machali.png (para: Control 2 Maitenes, Machalí, O'Higgins)
Código QR guardado como: qrcodes_waze/qr_codelco_quintero_puchuncavi.png (para: Quintero, Puchuncaví, Valparaíso (Codelco Ventanas))
Código QR guardado como: qrcodes_waze/qr_pelambres_shell_las_esteras_quilicura.png (para: Shell Las Esteras, Panamericana Norte 372, Quilicura, Región Metropolitana)
Código QR guardado como: qrcodes_waze/qr_pelambres_

In [10]:
pip install qrcode

In [8]:
pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.1 MB/s eta 0:00:00


In [9]:
pip install qrcode pillow reportlab

In [12]:
import qrcode
import os
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.lib.utils import ImageReader

# --- Lista de direcciones con coordenadas y detalles ---
# (Añadí campos 'lat' y 'lon' separados para facilitar la construcción del enlace)
direcciones = [
    # CODELCO
    {"empresa": "CODELCO", "nombre_base_archivo": "qr_gmaps_codelco_coya_h267",
     "direccion_original": "H-267 48, Coya, Machalí, O'Higgins",
     "lat": -34.0875, "lon": -70.5062,
     "coordenadas_texto": "-34.0875, -70.5062 (aprox. Tranques Barahona)"},

    {"empresa": "CODELCO", "nombre_base_archivo": "qr_gmaps_codelco_saladillo_los_andes",
     "direccion_original": "Saladillo, Los Andes, Valparaíso",
     "lat": -32.9264, "lon": -70.2600,
     "coordenadas_texto": "-32.9264, -70.2600 (aprox.)"},

    {"empresa": "CODELCO", "nombre_base_archivo": "qr_gmaps_codelco_garita_tove_tiltil",
     "direccion_original": "Garita Acceso TOVE, Tiltil, Región Metropolitana (Depósito Relaves Ovejería)",
     "lat": -33.041578, "lon": -70.798303,
     "coordenadas_texto": "-33.041578, -70.798303"},

    {"empresa": "CODELCO", "nombre_base_archivo": "qr_gmaps_codelco_embalse_caren_alhue",
     "direccion_original": "Embalse Carén, Alhué, Región Metropolitana",
     "lat": -34.107167, "lon": -71.139306,
     "coordenadas_texto": "-34.107167, -71.139306"},

    {"empresa": "CODELCO", "nombre_base_archivo": "qr_gmaps_codelco_control_maitenes_machali",
     "direccion_original": "Control 2 Maitenes, Machalí, O'Higgins",
     "lat": -34.193850, "lon": -70.563186,
     "coordenadas_texto": "-34.193850, -70.563186 (Acceso Principal)"},

    {"empresa": "CODELCO", "nombre_base_archivo": "qr_gmaps_codelco_quintero_puchuncavi",
     "direccion_original": "Quintero, Puchuncaví, Valparaíso (Codelco Ventanas)",
     "lat": -32.7470, "lon": -71.4842,
     "coordenadas_texto": "-32.7470, -71.4842 (aprox.)"},

    # PELAMBRES
    {"empresa": "PELAMBRES", "nombre_base_archivo": "qr_gmaps_pelambres_shell_las_esteras_quilicura",
     "direccion_original": "Shell Las Esteras, Panamericana Norte 372, Quilicura, Región Metropolitana",
     "lat": -33.3645, "lon": -70.7075,
     "coordenadas_texto": "-33.3645, -70.7075 (aprox. dirección, verificar Shell)"},

    {"empresa": "PELAMBRES", "nombre_base_archivo": "qr_gmaps_pelambres_shell_hijuelas_km108",
     "direccion_original": "Shell Hijuelas - Panamericana Norte Km. 108, Quillota Hijuelas, Valparaíso, Chile",
     "lat": -32.7982, "lon": -71.1591,
     "coordenadas_texto": "-32.7982, -71.1591"},

    {"empresa": "PELAMBRES", "nombre_base_archivo": "qr_gmaps_pelambres_copec_la_calera",
     "direccion_original": "Copec, La Calera (Copec Carrera)",
     "lat": -32.791633, "lon": -71.202731,
     "coordenadas_texto": "-32.791633, -71.202731"},

    {"empresa": "PELAMBRES", "nombre_base_archivo": "qr_gmaps_pelambres_copec_hijuelas_km10",
     "direccion_original": "COPEC HIJUELAS Carretera Panamericana Norte Km. 10 Ruta 5 Oriente, Hijuelas, Valparaíso",
     "lat": -32.8713, "lon": -71.0855,
     "coordenadas_texto": "-32.8713, -71.0855 (aprox.)"}
]

# --- Directorio para guardar los QRs ---
output_dir = "qrcodes_Maps"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- Generar y guardar cada código QR para Google Maps ---
print("Generando códigos QR para Google Maps...")
for item in direcciones:
    # Construir el enlace de Google Maps
    item["enlace_Maps"] = f"https://www.google.com/maps/search/?api=1&query={item['lat']},{item['lon']}"

    img = qrcode.make(item["enlace_Maps"])
    item["ruta_completa_qr"] = os.path.join(output_dir, item["nombre_base_archivo"] + ".png")
    img.save(item["ruta_completa_qr"])
    print(f"Código QR guardado como: {item['ruta_completa_qr']} (para: {item['direccion_original']})")
print("Proceso de generación de códigos QR completado.\n")


# --- Función para crear el PDF con los códigos QR ---
def create_pdf_with_qrcodes(pdf_filename="direcciones_Maps_qr.pdf", data_list=None):
    if data_list is None:
        data_list = []

    print(f"Creando PDF: {pdf_filename}...")
    c = canvas.Canvas(pdf_filename, pagesize=letter)
    width, height = letter

    margin_left = 0.75 * inch
    margin_top = 0.75 * inch
    margin_bottom = 0.75 * inch

    y_position = height - margin_top

    qr_size = 1.5 * inch
    line_height_small = 0.2 * inch
    line_height_medium = 0.3 * inch
    space_after_qr = 0.5 * inch

    for i, item in enumerate(data_list):
        block_height = (line_height_medium * 2) + (line_height_small * 2) + qr_size + space_after_qr

        if y_position - block_height < margin_bottom:
            c.showPage()
            y_position = height - margin_top

        c.setFont("Helvetica-Bold", 12)
        y_position -= line_height_medium
        c.drawString(margin_left, y_position, f"Empresa: {item['empresa']}")

        c.setFont("Helvetica", 10)
        y_position -= line_height_medium
        if len(item['direccion_original']) > 80:
            c.drawString(margin_left, y_position, item['direccion_original'][:80] + "...")
        else:
            c.drawString(margin_left, y_position, item['direccion_original'])

        y_position -= line_height_small
        c.drawString(margin_left, y_position, f"Coords: {item['coordenadas_texto']}")

        # Opcional: Mostrar el enlace de Google Maps
        # y_position -= line_height_small
        # c.setFont("Helvetica-Oblique", 7) # Fuente más pequeña para el link
        # if len(item['enlace_Maps']) > 90:
        #     c.drawString(margin_left, y_position, item['enlace_Maps'][:90] + "...")
        # else:
        #    c.drawString(margin_left, y_position, item['enlace_Maps'])
        # c.setFont("Helvetica", 10)

        y_position -= (qr_size + line_height_small)
        qr_image_path = item["ruta_completa_qr"]

        try:
            qr_img_reader = ImageReader(qr_image_path)
            c.drawImage(qr_img_reader, margin_left, y_position, width=qr_size, height=qr_size, preserveAspectRatio=True, anchor='nw')
        except Exception as e:
            print(f"Error al dibujar imagen {qr_image_path} en el PDF: {e}")
            error_y_pos = y_position + (qr_size / 2)
            c.drawString(margin_left, error_y_pos, f"Error al cargar QR: {os.path.basename(qr_image_path)}")

        y_position -= space_after_qr

        if i < len(data_list) - 1 :
             if y_position - (0.1 * inch) > margin_bottom:
                c.line(margin_left, y_position, width - margin_left - (0.5*inch) , y_position)
                y_position -= (0.2 * inch)

    c.save()
    print(f"PDF generado como: {pdf_filename}")

# --- Llamar a la función para crear el PDF ---
create_pdf_with_qrcodes(pdf_filename="CODELCO_PELAMBRES_QR_GoogleMaps.pdf", data_list=direcciones)

print("\nScript finalizado.")

Generando códigos QR para Google Maps...
Código QR guardado como: qrcodes_Maps/qr_gmaps_codelco_coya_h267.png (para: H-267 48, Coya, Machalí, O'Higgins)
Código QR guardado como: qrcodes_Maps/qr_gmaps_codelco_saladillo_los_andes.png (para: Saladillo, Los Andes, Valparaíso)
Código QR guardado como: qrcodes_Maps/qr_gmaps_codelco_garita_tove_tiltil.png (para: Garita Acceso TOVE, Tiltil, Región Metropolitana (Depósito Relaves Ovejería))
Código QR guardado como: qrcodes_Maps/qr_gmaps_codelco_embalse_caren_alhue.png (para: Embalse Carén, Alhué, Región Metropolitana)
Código QR guardado como: qrcodes_Maps/qr_gmaps_codelco_control_maitenes_machali.png (para: Control 2 Maitenes, Machalí, O'Higgins)
Código QR guardado como: qrcodes_Maps/qr_gmaps_codelco_quintero_puchuncavi.png (para: Quintero, Puchuncaví, Valparaíso (Codelco Ventanas))
Código QR guardado como: qrcodes_Maps/qr_gmaps_pelambres_shell_las_esteras_quilicura.png (para: Shell Las Esteras, Panamericana Norte 372, Quilicura, Región Metropo